In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from keras.layers import TimeDistributed
from keras.layers import Bidirectional

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc

Using TensorFlow backend.


importing Jupyter notebook from readWrite.ipynb
importing Jupyter notebook from vectorToDoc.ipynb
importing Jupyter notebook from dictionary.ipynb
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NielPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NielPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

Read complete, total records: 1125
Read complete, total records: 1125
x_train: 901, shape: (901, 20)
y_train: 901, shape: (901,)
x_test: 224, shape: (224, 20)
y_test: 224, shape: (224,)
encoding_train shape: (901, 3)
encoding_test shape: (224, 3)


In [13]:
vocabSize = 2000
outputDim = 64
recordsSize = 20

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(recordsSize, 1)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 64)            128000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20, 200)           132000    
_________________________________________________________________
dense_2 (Dense)              (None, 20, 3)             603       
Total params: 260,603
Trainable params: 260,603
Non-trainable params: 0
_________________________________________________________________


In [15]:
for idx, xdata in enumerate(x_train):    
    xdata = xdata.reshape(1, len(xdata))
    ydata = encoding_train[idx]
    ydata = ydata.reshape(20, len(ydata), 1)
    print(xdata.shape)
    print(ydata.shape)

    model.fit(xdata, ydata, epochs=1, batch_size=1, verbose=2)

ValueError: cannot reshape array of size 3 into shape (20,3,1)